In [1]:
import json

with open('documents-with-ids.json', 'rt') as f_input:
    documents = json.load(f_input)

In [2]:
from sentence_transformers import SentenceTransformer
model_name = "multi-qa-MiniLM-L6-cos-v1"
model = SentenceTransformer(model_name)

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
v = model.encode('I just discovered the course. Can i still join?')

In [4]:
len(v)

384

In [5]:
v.dot(v)

np.float32(1.0)

In [6]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

index_setting = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id":{"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity":"cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity":"cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity":"cosine"
            },
        }
    }
}

index_name = "course_questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_setting)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_questions'})

In [7]:
from tqdm.auto import tqdm

In [8]:
for doc in tqdm(documents):
    questions = doc['question']
    text = doc['text']
    qt = questions + ' ' + text

    doc['question_vector'] = model.encode(questions)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [01:55<00:00,  8.23it/s]


In [9]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:20<00:00, 47.11it/s]


In [10]:
query = "I just discovered the course. Can I still join it?"

In [11]:
v_q = model.encode(query)

In [12]:
search_query = {
    "field" : "question_vector",
    "query_vector": v_q,
    "k" : 5,
    "num_candidates" : 10000,
}

es_results = es_client.search(
    index=index_name,
    knn=search_query,
    source=["text", "section", "question", "course", "id"]
)

In [13]:
result_docs = []

for hit in es_results['hits']['hits']:
    result_docs.append(hit['_source'])

In [14]:
result_docs

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'id': '7842b56a'},
 {'question': 'Course

In [15]:
search_query = {
    "knn": {
        "field" : "question_vector",
        "query_vector": v_q,
        "k" : 5,
        "num_candidates" : 10000,
        "filter": {
            "term": {
                "course": "data-engineering-zoomcamp"
                
            }
        }
    },
    "_source": ["text", "section", "question", "course", "id"]
        
}

es_results = es_client.search(
    index=index_name,
    body=search_query
)


In [16]:
result_docs = []

for hit in es_results["hits"]["hits"]:
    result_docs.append(hit['_source'])


In [17]:
result_docs

[{'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'id': '7842b56a'},
 {'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'id': 'a482086d'},
 {'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp',
  'section': '

In [26]:
def elastic_search_knn(field, vector, course):
        knn = {
            "field" : field,
            "query_vector": vector,
            "k" : 5,
            "num_candidates" : 10000,
            "filter": {
                "term": {
                    "course": course
                
                }
            }
        }


        search_query = {
            "knn": knn,
            "_source": ["text", "section", "question", "course", "id"]
        
        }

        es_results = es_client.search(
            index=index_name,
            body=search_query
    )

        result_docs = []

        for hit in es_results["hits"]["hits"]:
            result_docs.append(hit['_source'])

        return result_docs


In [27]:
elastic_search_knn('question_vector', v_q, 'data-engineering-zoomcamp')

[{'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'id': '7842b56a'},
 {'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'id': 'a482086d'},
 {'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp',
  'section': '

In [35]:
def question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_vector', v_q, course)

In [28]:
import pandas as pd

In [29]:
df_ground_truth = pd.read_csv("ground-truth-data.csv")
ground_truth = df_ground_truth.to_dict(orient='records')

In [30]:
ground_truth[0]

{'question': 'When does the course begin?',
 'course': 'data-engineering-zoomcamp',
 'document': 'c02e79ef'}

In [31]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1
    return cnt / len(relevance_total)

In [32]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank +1)
    return total_score / len(relevance_total)

In [33]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }
        

In [36]:
evaluate(ground_truth, question_vector_knn)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 4627/4627 [01:45<00:00, 43.75it/s]


{'hit_rate': 0.773071104387292, 'mrr': 0.6666810748505158}

In [38]:
def text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('text_vector', v_q, course)

In [39]:
evaluate(ground_truth, text_vector_knn)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 4627/4627 [01:40<00:00, 45.94it/s]


{'hit_rate': 0.8286146531229739, 'mrr': 0.7062315395144454}

In [40]:
def question_text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course)

In [41]:
evaluate(ground_truth, question_text_vector_knn)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 4627/4627 [01:37<00:00, 47.53it/s]


{'hit_rate': 0.9172249837907932, 'mrr': 0.824306606152295}